In [8]:
from pathlib import Path

import pandas as pd

from tqdm import tqdm
from rdkit import Chem

In [19]:
zinc_path = Path('..') / 'FinalPipeline' / 'generated_mols' / 'scored' / 'scored_zinc.csv'


In [20]:
zinc_df = pd.read_csv(zinc_path)

In [21]:
zinc_df

,smiles,SA,logP,QED,weight,n_rings
0,CC(C)(C)c1ccc2occ(CC(=O)Nc3ccccc3F)c2c1,2.084095,5.05060,0.731901,325.383,3
1,CC1CC(C)CC(Nc2cncc(-c3nncn3C)c2)C1,3.432004,3.11370,0.941112,285.395,3
2,N#Cc1ccc(-c2ccc(OC(C(=O)N3CCCC3)c3ccccc3)cc2)cc1,2.470633,4.96778,0.626105,382.463,4
3,CCOC(=O)C1CCCN(C(=O)c2nc(-c3ccc(C)cc3)n3c2CCCC...,2.822753,4.00022,0.716225,409.530,4
4,N#CC1=C(SCC(=O)Nc2cccc(Cl)c2)N=C([O-])C(C#N)C1...,4.035182,3.60956,0.809572,413.910,3
...,...,...,...,...,...,...
249450,CC1(C)CCC(CNC(=O)Cn2ncc3ccccc3c2=O)c2ccccc21,2.900726,3.36790,0.760407,375.472,4
249451,Cn1ccnc1C(=O)c1ccc(NC(=O)C2CCN(C(=O)C(C)(C)C)C...,2.326627,2.87430,0.805660,396.491,3
249452,Cc1ccc(NC(=O)C(=O)N(C)Cc2ccccc2)c(C)c1,1.840642,2.90054,0.885407,296.370,2
249453,Cc1cc(C(=O)Nc2ccc(OCC(N)=O)cc2)c(C)n1C1CC1,2.024638,2.55624,0.854652,327.384,3


In [22]:
def remove_charged_atoms(df):
    # List to keep track of the indices to be removed
    indices_to_remove = []
    
    # Iterate through the DataFrame
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        smiles = row['smiles']
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:  # invalid SMILES string
            continue

        for atom in mol.GetAtoms():
            if (atom.GetSymbol() == 'C' and atom.GetFormalCharge() < 0) or \
               (atom.GetSymbol() == 'O' and atom.GetFormalCharge() > 0) or \
               (atom.GetSymbol() == 'S' and atom.GetFormalCharge() > 0):
                indices_to_remove.append(index)
                break
                
    # Drop the rows with the indices in indices_to_remove
    return df.drop(indices_to_remove)


def remove_P(df):
    df = df[df['smiles'].str.contains('P') == False]
    return df

In [23]:
filtered_df = remove_charged_atoms(remove_P(zinc_df))

100%|██████████| 249332/249332 [01:41<00:00, 2455.30it/s]


In [24]:
filtered_df.to_csv(Path('..') / 'FinalPipeline' / 'generated_mols' / 'scored' / 'scored_zinc_filtered.csv', index=False)